In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset.idDataModule import IdDataModule
from model.idTransformerModel import IdTransformerModel
import hydra
from omegaconf import OmegaConf
from tqdm.auto import trange, tqdm
import warnings
import torch
from torch.utils.data import DataLoader
warnings.filterwarnings("ignore")

In [3]:
conf = OmegaConf.load('configs/config.yaml')
conf['model'] = OmegaConf.load('configs/model/transformer_encoder.yaml')
conf.model.batch_size = 1
conf.model.max_num_usages = None
conf['dataset'] = OmegaConf.load('configs/dataset/java-small.yaml')
conf.dataset.usage_vocab_min_freq = 3
conf.dataset.target_vocab_min_freq = 2
def dm(cfg):
    datamodule = IdDataModule(cfg)
    datamodule.setup('fit')
    return datamodule

dm = dm(conf)

Building train dataset...
Building val dataset...
Building test dataset...
Loading usage counter from C:\Users\Igor\IdeaProjects\ide-plugin\build\idea-sandbox\system\dataset\java-small\usage_vocab.json
Loading target counter from C:\Users\Igor\IdeaProjects\ide-plugin\build\idea-sandbox\system\dataset\java-small\target_vocab.json

Usage vocabulary size is 23418
Target vocabulary size is 14377


In [4]:
model = IdTransformerModel.load_from_checkpoint(r"checkpoints/11.03-transformer_encoder-java-small-epoch=02-val_accuracy=0.19.ckpt",
                                               map_location="cpu",
                                               dm=dm,
                                               cfg=conf.model)

In [5]:
vocab = dm.dataset.target_vocabulary
def itos(ids):
    ids.reshape(-1)
    return list(map(lambda x: vocab.itos[x], ids))

In [6]:
_ = model.train(False)

In [17]:
# torch.manual_seed(42)
iterator = iter(DataLoader(dm.dataset.test,
                                       batch_size=1,
                                       shuffle=True,
                                       collate_fn=dm.dataset.collate_fn))

In [74]:
ex = next(iterator)
ex


[torchtext.data.batch.Batch of size 1]
	[.target]:('[torch.LongTensor of size 6x1]', '[torch.LongTensor of size 1]')
	[.usages]:('[torch.LongTensor of size 1x5x41]', '[torch.LongTensor of size 1]', '[torch.LongTensor of size 1x5]')

In [75]:
itos(ex.target[0])

['<s>', 'pending', 'event', 'queue', '</s>', '<pad>']

In [79]:
sum_p = 0
for pred in model(ex.usages[0]):
    sum_p += pred[1].p
    print(f"{itos(pred[1].token_ids)}: {pred[1].p}")
print(sum_p)

['<s>', 'queue', '</s>']: 0.3604006853478108
['<s>', '<unk>', '</s>']: 0.020605530450632493
['<s>', 'event', 'queue', '</s>']: 0.017151209875754917
['<s>', 'pending', 'queue', '</s>']: 0.014578924691573953
['<s>', 'queue', 'queue', '</s>']: 0.011628744575718138
['<s>', 'schedule', 'queue', '</s>']: 0.01110532901054162
['<s>', 'waiting', 'queue', '</s>']: 0.009525916241712649
['<s>', 'work', 'queue', '</s>']: 0.008030421008509207
['<s>', 'locks', '</s>']: 0.005735279604092989
['<s>', 'clients', '</s>']: 0.004684474300920446
0.46344651510726725


In [35]:
%timeit model(ex.usages[0])

156 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
